# Lifetimes, Annotations & Internal mutablility

## Types of &str

* **String literals** - they last for the whole program (they are written directly into the binary)
  * type - `&'static str`

In [2]:
let name: &'static str = "John Smith";

* **Borrowed `str`** - borrowed reference to `String` object (without `'static` lifetime)
  * conversion from `String` to `&str` is allowed thanks to a `Deref` trait
  * we can always pass reference to `String` to a function that expects `&str` - it will be automatically converted (*type coercion* - *deref coercion*)

In [3]:
fn print_description(description: &str) {
    println!("{}", description);
}

fn main() {
    let description: String = "And now something completly different...".to_string();
    let borrowed_description: &str = &description; 

    print_description(&description); // deref coercion works here
}

main();

And now something completly different...


## Returning references

* You can’t return a reference to local variable that only lives inside a function - it would be a *dangling reference*

In [4]:
fn create_dangling_reference() -> &'static str {
    let s = String::from("Hello, world!");
    &s
}

Error: unused variable: `borrowed_description`

Error: cannot return reference to local variable `s`

* You can return a reference to static variable (like string literals) - return type needs `'static` *lifetime annotation*

In [5]:
fn get_greetings() -> &'static str {
    let s = "Hello, world!";
    s
}

## Lifetime annotations

* Most of the time, Rust takes care of lifetimes for you, but sometimes, it needs a bit of extra help. This extra help is called a **lifetime annotation**, which means “extra lifetime information.”
* Liftetime annotations don't change the lifetimes of any references, they just describe the relationships of the lifetimes of multiple references.
* Lifetime annotations syntax: the names of lifetime parameters must start with an apostrophe (') and are usually all lowercase and very short, like this: `'a`, `'b`, and so on. But the names of lifetime parameters are arbitrary and have no meaning.

### Inferring lifetimes

* Rust has a **lifetime elision** rules, which are a set of three rules that the compiler follows to figure out what lifetimes references have when there aren't explicit annotations.
* The first rule applies to input lifetimes, and the second and third rules apply to output lifetimes. If the compiler gets to the end of the three rules and there are still references for which it can’t figure out lifetimes, the compiler will stop with an error.

* **The first rule**: each parameter that is a reference gets its own lifetime parameter. In other words, a function with one parameter gets one lifetime parameter: `fn foo<'a>(x: &'a i32);` a function with two parameters gets two separate lifetime parameters: `fn foo<'a, 'b>(x: &'a i32, y: &'b i32);` and so on.
* **The second rule**: if there is exactly one input lifetime parameter, that lifetime is assigned to all output lifetime parameters: `fn foo<'a>(x: &'a i32) -> &'a i32;`
* **The third rule**: if there are multiple input lifetime parameters, but one of them is `&self` or `&mut self` because this is a method, the lifetime of `self` is assigned to all output lifetime parameters: `fn foo<'a, 'b>(&'a self, x: &'b i32) -> &'a i32;`

In [6]:
fn foo(text: &str) {
    println!("{}", text);
}

can be explicitly written as:

In [7]:
fn foo<'a>(text: &'a str) {
    println!("{}", text);
}

let text = String::from("Hello, world!");
foo(&text);

Hello, world!


From the `foo` signature Rust knows it will not save `text` anywhere that might outlive the function call: any lifetime that encloses the call must work for `'a`.

In [8]:
static mut BAG: Vec<&str> = Vec::<&str>::new();

fn bar<'a>(text: &'a str) {
    unsafe {
        BAG.push(text);
    }
}

fn main() {
    let text = String::from("Hello, world!");
    bar(&text);
}

Error: borrowed data escapes outside of function

Acceptable lifetime for text is `static`:

In [9]:
static mut BAG: Vec<&str> = Vec::<&str>::new();

fn bar(text: &'static str) {
    unsafe {
        BAG.push(text);
    }
}

fn main() {
    let text = "Hello, world!";
    bar(&text);
}

### Returning references

When a function takes a single reference as an argument, and returns a single reference, Rust assumes that the two must have the same lifetime (see *the second rule*).

```rust

In [10]:
fn ref_to_min(data: &mut [i32]) -> &mut i32 {
    // let min = data.iter_mut().min().unwrap();
    
    let n = data.len();
    let mut index: usize = 0;
    for (i, value) in data.iter().enumerate() {
        if *value < data[index] {
            index = i;
        }
    } 

    &mut data[index]
}

In [11]:
fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);
    *the_smallest = 0;

    println!("{:?}", data);
}

main();

[42, 665, 0, 44, 99]


In [12]:
let s;

{
    let mut data = [9, 4, 1, 0, 1, 4, 9];
    s = ref_to_min(&mut data);
}

println!("{}", *s);

Error: unused variable: `n`

Error: `data` does not live long enough

## Structs Containing References (Annotations)

In [13]:
struct View {
    value: &i32
}

fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);

    let view = View { value: the_smallest };
    println!("{}", view.value);
}

Error: missing lifetime specifier

Error: unused variable: `n`

Whenever a reference type appears inside another type’s definition, you must write out its lifetime.

In [14]:
struct View<'a> {
    value: &'a i32
}

fn main() {
    let mut data = [42, 665, 1, 44, 99];
    let the_smallest: &mut i32 = ref_to_min(&mut data);
    let view = View { value: the_smallest };
    println!("{}", view.value);
}

main();

1


Now the `View` type has a lifetime, just as reference types do. Each value you create of type `Value` gets a fresh lifetime `'a`, which becomes constrained by how you use the value. The lifetime of any reference you store in `ref_value` had better enclose `'a`, and `'a` must outlast the lifetime of wherever you store the `View`.

In [15]:
struct View<'a> {
    value: &'a i32
}

fn main() {
    let mut bag: Vec<View> = Vec::new();
    {
        let mut data = [42, 665, 1, 44, 99];
        let the_smallest: &mut i32 = ref_to_min(&mut data);

        let view = View { ref_value: the_smallest };
        println!("{}", view.ref_value);
        bag.push(view);
    }

    println!("{}", bag[0].ref_value);
}

main();

Error: struct `View<'_>` has no field named `ref_value`

Error: no field `ref_value` on type `View<'_>`

Error: no field `ref_value` on type `View<'_>`

Error: unused variable: `n`

When we want to place `View` inside of another `struct`, we must specify the lifetime of the reference it contains:

In [17]:
struct Record {
    v: View<'static>
}

// or

struct MinView<'a> {
    v: View<'a>
}

A type’s lifetime parameters always reveal whether it contains references with interesting (that is, non-`'static`) lifetimes, and what those lifetimes can be

In [20]:
fn get_min<'a>(data: &'a mut [i32]) -> MinView<'a> {
    let min: &'a mut i32 = data.iter_mut().min().unwrap();
    MinView { v: View { value: min } }
}

Without looking into the definition of the `MinView` type at all, we can tell that, if we receive a `MinView` from `get_min`, whatever references it contains must point into the input slice we passed in, and nowhere else (except perhaps at 'static values).